Markdown cells are for personal notes & thought process. This notebook in general is for a more interactive experience as I work through setting up the database for this helper tool.

In [53]:
import pandas as pd
import numpy as np
import os, re

In [3]:
os.listdir()

['wordnet20-from-prolog-all-3.sql',
 'Untitled.ipynb',
 'cmudict-0.7b.phones',
 'cmudict-0.7b',
 'newdic.txt',
 '.ipynb_checkpoints',
 'cmudict-pared.txt',
 'cmudict-0.7b.symbols']

## Initial data import
file - 'cmu-dict-test.txt', temporary project file with a much smaller overall list of words

issues: 
-parse the two columns to a dataframe, pronunciations are separated by white space, each word is separated by a new line.
-supply column names

word -> first white space -> turn pronunciations in to list
    -i.e. HH A0 R D L YY1 ['HH', 'A0', 'R', 'D', 'L', 'YY1']
    -then look for \n for next entry in dataframe
    -to do this, we can make an initial list by removing all whitespace, index 0 is the word. the rest will be the phonemes.
    
potential steps:
- make a list of each word, then each pronunciation
- after this, I have my basis for the initial dataframe
- then I can write functions to parse each pronunciation for syllable count & stressed, and create a list for each of those

In [34]:
dictionary = open('cmudict-pared.txt', 'r', encoding="ISO-8859-1")
# utf-8 was not the default encoding for this file. 
# something to keep note of in case this affects searching results
# later in the project

In [35]:
# dictionary.readline()
# imported correctly, now to make initial list, per line in the file

In [84]:
# each word in the CMUdict is separated by a new line.
# want to split and get rid of \n character

entries = [word.rstrip('\n') for word in dictionary]

In [83]:
# checking random entries

entries[101]

'ABBENHAUS  AE1 B AH0 N HH AW2 S'

### splitting each entry to a word & phonics

- must first split each by white space and then pull index 0 from each of those splits.

In [42]:
sample_word = entries[0].split(' ')
sample_word

["'ALLO", '', 'AA2', 'L', 'OW1']

In [44]:
# set up where they'll go, and we can probably separate the word
# & pronunciation lists in one pass while using the split method.
words = []
pronunciations = []

for item in entries:
    item = item.split(' ')
    words.append(item[0])
    pronunciations.append(item[1:])

In [46]:
len(words)

134323

In [47]:
len(pronunciations)

134323

In [48]:
words[13]

"'ROUND"

In [63]:
pronunciations[45]

['', 'AA2', 'HH', 'UW1', 'S']

* something to keep track of is the pronunciation in list format. When searching for rhymes, would it be more efficient to have this in list form or string form? It would be a matter of comparing substrings and delimited white space vs number of indexes in common at the end of each list.

## getting syllable counts

In [64]:
count = 0
numbers = r'\d'

# count = sum(numbers in item for item in pronunciations[13])
# count

# this is a potential setup for what I need to do for syllable counts
# a list comprehension with something like the below could get me syllable
# counts for each word.

for item in pronunciations[45]:
    if bool(re.search(numbers, item)):
        count += 1
count

2

In [66]:
def count_syllables(list):
    count = 0
    for item in list:
        if bool(re.search(numbers, item)):
            count += 1
    return count

In [67]:
syllables = [count_syllables(x) for x in pronunciations]

In [68]:
len(syllables)

134323

^ looks like we got the whole list

\\/ some checks for various syllable counts

In [69]:
syllables[45]

2

In [82]:
syllables[197]

5

In [81]:
words.index('ABERRATIONAL')

197

Looking good, now to parse meter from each of these entries.

## Syllable stresses

The numbers attached to vowel sounds indicate stresses;
- 0: no stress
- 1: primary stress
- 2: secondary stress

### How to handle stresses?
- secondary stresses have a weird place in this project. 
- I could decide to count them as stresses always, but that doesn't tell the whole story of secondary stresses.
- There are cases where secondary stresses get promoted to a primary stress depending on surrounding syllables and/or meter.
- For the set-up of my database, more research on how to handle these will need to be.
- In the end it will modify how some functions are written and certainly how lines are parsed for meter when that feature is added.

Will need to look at the necessity of separating primary & secondary stress vs combining them... '\`_

... So why make all these lists and not just run the functions each time a word is searched?
    - It'll be better to search the database pre-filled later than run functions over the entire dictionary later.
    - The database may need further fields, or bins of some kind for feet/meter & rhymes. Those will require searching the entire dicitonary.
    - English has 44 phonemes. These pronunciations can be grouped by phonemes. ??? searching end-rhymes vs full/partial word-rhymes???